### Import Library

In [2]:
%matplotlib inline
import matplotlib.pyplot as plt
plt.rcParams['axes.unicode_minus'] = False
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.metrics import mean_squared_error,mean_absolute_error
from sklearn.linear_model import Lasso

import sys
sys.path.insert(1, "/Users/yeojisu/Documents/House-Price-Index-Prediction")

from utils.data_loader import *
from config.config_KR import *

### Load Data

In [ ]:
target_df = load_target_data(TARGET_PATH)
df = load_feature_data(FEATURE_PATH, FEATURE_COLUMN, TRAIN_DATE)

df = df.shift(FORECAST_HORIZON) # Feature - shift 3
for col in FEATURE_COLUMN:
    df[f'{col}_lag{FORECAST_HORIZON+1}'] = df[col].shift(FORECAST_HORIZON+1) # Feature - shift 4
for col in FEATURE_COLUMN:
    df[f'{col}_lag{FORECAST_HORIZON+2}'] = df[col].shift(FORECAST_HORIZON+2) # Feature - shift 5
for col in target_df.columns:
    df[f'{col}_lag{FORECAST_HORIZON}'] = target_df[col].shift(FORECAST_HORIZON) # Target - shift 3
for col in target_df.columns:
    df[f'{col}_lag{FORECAST_HORIZON+1}'] = target_df[col].shift(FORECAST_HORIZON+1) # Target - shift 4
for col in target_df.columns:
    df[f'{col}_lag{FORECAST_HORIZON+2}'] = target_df[col].shift(FORECAST_HORIZON+2) # Target - shift 5
    
df = pd.concat([df,target_df], axis = 1)
for col in df.columns:
    df[col] = pd.to_numeric(df[col], errors='coerce')
df = df.dropna()

### Train and forecast

In [ ]:
model_name = "Lasso"
dir_path = f"/Users/yeojisu/Documents/HPI-Save/Korea/Baseline/traditional/{FORECAST_HORIZON}_{model_name}.csv"

for target in target_df.columns:
    x_df = df.drop(target_df.columns,axis=1)
    y_df = df[target]

    # Split Train and Test Data
    X_train, X_test, y_train, y_test = x_df[x_df.index<TEST_DATE], x_df[x_df.index>=TEST_DATE], y_df[y_df.index<TEST_DATE], y_df[y_df.index>=TEST_DATE]

    # Scaling
    x_df_mu, x_df_std = X_train.mean(), X_train.std()
    y_df_mu, y_df_std = y_train.mean(), y_train.std()

    X_train = z_transform(X_train, x_df_mu, x_df_std)
    X_test = z_transform(X_test, x_df_mu, x_df_std)
    y_train = z_transform(y_train, y_df_mu, y_df_std)
    y_test = z_transform(y_test, y_df_mu, y_df_std)

    df_train_inv = pd.concat([pd.DataFrame(y_train), X_train], axis=1)
    df_test_inv = pd.concat([pd.DataFrame(y_test), X_test], axis=1)
    
    model = Lasso(alpha=0.1)
    model.fit(X=X_train, y=y_train)
    pred_z = model.predict(X_test)
    pred = inv_z_transform(pred_z, y_df_mu, y_df_std)
    real = inv_z_transform(y_test, y_df_mu, y_df_std)
    rmse = round(mean_squared_error(real, pred)**0.5,3)
    mae = round(mean_absolute_error(real, pred),3)
    val =[target.split("_")[0],rmse,mae]
    
    pd.DataFrame(val).T.to_csv(dir_path, mode='a', header=False, index=False)
    